# Agrégateur Web

Grâce à Python, on peut extraire des informations d'un site web et les structurer pour en faire l'analyse plus facilement. L'agrégation web (web scraping en anglais) est une habileté qui vaut la peine d'apprendre pour pouvoir extraire le texte, donées, et telécharger les fichiers de toute sortes de sites.

Nous utiliserons le site web de la ville de Montréal: https://donnees.montreal.ca/ville-de-montreal comme source pour cet atelier.

### Questions:
- Quels sont les jeux de données disponibles en rapport à l'eau de Montréal ?
  (ceci correspond aux résultats de la recherche pour le `tag` [Eau](https://donnees.montreal.ca/search?q=tags:Eau))
- Quels sont les formats de fichiers offerts ?

### Libraries:
Nous allows utiliser:
- le module `requests` pour fire les requètes HTTP
- le module Beautiful Soup (`bs4`) pour acceder au contenu des page web de façon structurée

## 1. Téléchargement de la page

Nous voulons télécharger le contenu de la page suivante: https://donnees.montreal.ca/search?q=tags=Eau

Pour ce faire, nous utiliserons le module `requests` qui agit un peu comme votre furteur web, ça GET des pages web.

In [ ]:
import requests

HOST = "https://donnees.montreal.ca"
PATH = "/search"
QUERY_STRING = "?q=tags=Eau"
EAU_URL = HOST + PATH + QUERY_STRING

response = requests.get(EAU_URL)

In [ ]:
type(response)

In [ ]:
len(response.text)

In [ ]:
# premiers 100 charactères du contenu de la page
print(response.text[0:100])

In [ ]:
# le contenu de la page au complet
# response.text

Nou avons le contenu de la [page web](https://donnees.montreal.ca/search?q=tags=Eau) sous forme de text dans `response.text`, mais ce n'est pas très utile par soi.

## 2. Analyse structurelle du contenu HTML de la page

La librairie Python Beautiful Soup nous permet de comprendre la structure de la page HTML ce qui est très utile.

In [ ]:
import bs4
soup = bs4.BeautifulSoup(response.text)

In [ ]:
type(soup)

Voici un aperçu de comment on peut accéder aux éléments dans notre soupe.

In [ ]:
# imprimer le contenu du premier lien dans la page
html = soup.find("html")
link = html.find("a")
print(link)

In [ ]:
# trouver le premier titre H3 dans le grand-grand-grand-parent du premier lien
titre_h3 = link.parent.parent.parent.parent.find("h3")
print("text =", titre_h3.text)
print("len(text) =", len(titre_h3))
print("CSS classes =", titre_h3["class"])
print("children=", list(titre_h3.children))

On voit qu'on peut naviguer le document HTML comme un arbre avec les attributs `parent` et `children`. On voit aussi qu'on peut trouver des éléments spécifiques par nom de balises (en anglais: "tag").

Vous pouvez imaginer une page Web comme un arbre de balises. Prennont le document suivant:

In [ ]:
html1 = """
<html>
  <head>
    <title>Titre de la page</title>
  </head>
  <body>
    <h1>Gros Titre de Section</h1>
    <p>Premier paragraphe de la section</p>
    <p>Second pragraphe de la section</p>
  </body>
</html>
"""

On peut facilement l'imaginer comme une série de liste imbriquées, mais on peut aussi voir la relation conteneur/conetnu comme une relation parent/enfant.

![Arbre de éléments HTML](https://github.com/mtlpy/ateliers-prog/raw/main/assets/html-as-tree.png)

In [ ]:
# pour pouvoir traverser l'arbre de balises...
soup1 = bs4.BeautifulSoup(html1)

In [ ]:
# texte_1
soup1.head.title.text

In [ ]:
# texte_2
soup1.body.h1.text

In [ ]:
# texte_3
soup1.body.find_all("p")[0].text

In [ ]:
# texte_4
soup1.body.find_all("p")[1].text

Pour ce qui suit, on va appeler "élément" le bloc complet qui commence à la balise ouvrante et qui se termine à la fin de la balise fermante.  L'élément peut posséder des attributs comme `id` et `class` qu'on accède avec la notation des dictionnaires Python. Les enfants d'un élément peuvent être des blocs de texte ou d'autres éléments.

Pour mieux comprendre, visitez la même page web et faites "inspect element" sur un des titres de sections dans votre navigateur. Notez le parallèle entre le code de la page web et la vue offerte par BeautifulSoup.  Si vous n'avez pas l'option "inspect element", installez l'extension "Web Developer" dans Firefox ou dans Chrome faites clique contextuel et choisissez "Inspect".

## 3. Sélection d'éléments avec la notation CSS

On peut prendre un raccourci pour trouver les éléments dans une page Web avec la méthode `select()` qui prend en argument un sélecteur CSS. La [notation des sélecteurs CSS](https://en.wikipedia.org/wiki/CSS#Selector) est très expressive et très compacte.

Par exemple, considéron la page web suivante:

In [ ]:
html2 = """
<html>
  <body>
    <h1 class="big">Gros Titre de Section</h1>
    <p id="first_para">Premier paragraphe de la section</p>
    <p id="second_para">Second pragraphe de la section</p>
  </body>
</html>
"""
soup2 = bs4.BeautifulSoup(html2)

Dans la notation des sélecteurs CSS, on peut référencer des élements par leur imbracage (par exemple `"body h1"`), par leur classes (ex.: `"h1.big"`), par leur identifiant (ex.: `"p#first_para"`) ou une combinaison arbitraire de ces options.

In [ ]:
# exemple: selectionner le premier paragraphe
soup2.select("#first_para")[0]

In [ ]:
# à vous de jouer : selectionnez le texte du second paragraphe


$ $

---



Dans le contexte de notre [page web](https://donnees.montreal.ca/search?q=tags=Eau), on peut utiliser les selecteurs CSS suivants:

In [ ]:
# selectionner tous le titres de jeux de données
# tous les éléments <a> qui sont enfants d'éléments <h3> avec la classe "text-lg"
soup.select("h3.text-lg a")

____

## 4. Exercices

Rappelez vous que `soup` est l'objet BeautifulSoup crée à partid du code HTML de la [page web](https://donnees.montreal.ca/search?q=tags=Eau).

```Python
response = requests.get(EAU_URL)
soup = bs4.BeautifulSoup(response.text)
```

### Exercice 1: liste des jeux de données

Utilisez la méthode `select()` pour extraire la liste des jeux de données. Affichez les résultats dans les format "titre: url". Par exemple:

    Bornes d'incendie: https://donnees.montreal.ca/...
    Hydrographie: https://donnees.montreal.ca/...
    ...[link text](https://)

In [ ]:
# tapper votre solution ici
# indice: soup.???


### Exercice 2: formats de fichier disponibles

Modifiez le code précédent pour inclure les formats disponibles avant l'URL. Par exemple:

    Bornes d'incendie (JSON,SHP): https://donnees.montreal.ca/...
    Hydrographie (SHP,KML,PDF): https://donnees.montreal.ca/...
    ...

In [ ]:
# tapper votre solution ici


### Exercice 3: pagination

Trouvez sur combien de pages les résultats sont répartis, téléchargez toutes les pages, puis combinez les résultats dans une seule liste.

Par exemple:

    Bornes d'incendie (JSON, SHP): https://donnees.montreal.ca/...
    Hydrographie (SHP, KML, PDF): https://donnees.montreal.ca/...
    ...
    Regards d'égouts (SHP, GeoJSON): https://donnees.montreal.ca/...
    ...

In [ ]:
# tapper votre solution ici


# Autres resources

Sites pour pratiquer:

- Livres ou quotes: http://toscrape.com/
- Pays du monde: https://www.scrapethissite.com/pages/simple/


Exemples:
- [Tutoriel video](http://35.196.115.213/en/learn/#/topics/c/73470ad1a3015769ace455fbfdf17d48) qui montre comment télécharger un [site de contenu éducatif](https://shls.rescue.org/). Voir [ici](https://github.com/learningequality/sushi-chef-shls/blob/master/sushichef.py#L226-L336) pour le code final.

# License

Copyright 2021–2023 Montréal-Python

Permission is hereby granted, free of charge, to any person obtaining a copy of this software and associated documentation files (the "Software"), to deal in the Software without restriction, including without limitation the rights to use, copy, modify, merge, publish, distribute, sublicense, and/or sell copies of the Software, and to permit persons to whom the Software is furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.
